# Qual a média de nota por aluno e trimestre?

In [21]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, inspect 

In [22]:
engine = create_engine('sqlite:///:memory:') # Criando banco SQL
engine 

Engine(sqlite:///:memory:)

In [23]:
tabela_notas = pd.read_excel('files/tabela_nota.xlsx') # Importando tabela de notas
tabela_notas.drop(tabela_notas.loc[tabela_notas['id_materia']==0].index, inplace=True) # Não é possivel saber qual materia ele fez
tabela_notas['id'] = range(tabela_notas.shape[0])
tabela_notas.set_index('id', inplace=True)
tabela_notas.index = tabela_notas.index + 1
tabela_notas.to_sql('notas', engine) 
inspector = inspect(engine) 
print(inspector.get_table_names())

['notas']


In [31]:
tabela_alunos = pd.read_excel('files/tabela_estudante.xlsx', index_col=False) # Importando tabela de alunos
tabela_alunos.set_index('Unnamed: 0', inplace=True)
tabela_alunos.index.rename('id', inplace=True)

## Média de nota por aluno

In [53]:
# Função que seleciona o aluno para fazer a media das notas
def seleciona_aluno(nome: str) -> int:
    aluno = tabela_alunos[tabela_alunos['nome'] == nome].index
    aluno = aluno[0]
    return aluno

# Funcao que busca a tabela de notas do aluno selecionado
def busca_tabela_notas_aluno(nome_aluno: str):
    query = f'select * from notas where id_estudante = {seleciona_aluno(nome_aluno)}'
    notas_aluno_selecionado = pd.read_sql(query, engine)
    return notas_aluno_selecionado

# Pega o nome do aluno selecionado
def traz_nome_aluno(id_aluno: int) -> str:
    nome_aluno = tabela_alunos['nome'][id_aluno]
    return nome_aluno
    
# Funcao para calcular media das notas de cada aluno
def media_notas_aluno_selecionado(nome_aluno: str) -> float:
    notas_aluno_selecionado = busca_tabela_notas_aluno(nome_aluno)
    media_notas = notas_aluno_selecionado['nota'].mean()
    return media_notas

# Exportar no excel
def exporta_para_excel(itens: list, nome_coluna_itens: str, valores: list, nome_coluna_valores: str, caminho: str):
    dados = pd.DataFrame({nome_coluna_itens: itens, nome_coluna_valores: valores})
    dados.to_excel(caminho, index=False)

In [50]:
media_nota_dos_alunos = []
qtd_alunos = tabela_alunos.shape[0] + 1

for i in range(1, qtd_alunos):
    media_nota_dos_alunos.append(media_notas_aluno_selecionado(traz_nome_aluno(i)))

In [51]:
estudantes = list(tabela_alunos['nome'])

In [52]:
exporta_para_excel(estudantes, 'Estudante', media_nota_dos_alunos, 'Média das Notas', 'files/files_questions/question_2/media_por_estudante.xlsx')